In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt
import seaborn as sns
import re
from bs4 import BeautifulSoup
import time
import random
import os

In [ ]:
url = "/content/drive/MyDrive/aiffel_final_project/data_renew/aiffel_book.json"

with open(url,"r",encoding="utf-8") as f:
    data = json.load(f)

In [ ]:
data = pd.DataFrame(data)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ISBN         50000 non-null  object 
 1   ITEM_ID      50000 non-null  int64  
 2   BID          42652 non-null  float64
 3   GOODS_NO     7348 non-null   float64
 4   분류           46967 non-null  object 
 5   제목           49996 non-null  object 
 6   부제           33406 non-null  object 
 7   원제           4506 non-null   object 
 8   저자           50000 non-null  object 
 9   발행자          50000 non-null  object 
 10  발행일          50000 non-null  object 
 11  페이지          50000 non-null  int64  
 12  가격           50000 non-null  int64  
 13  표지           50000 non-null  object 
 14  간략소개         48590 non-null  object 
 15  책소개          47002 non-null  object 
 16  저자소개         37934 non-null  object 
 17  목차           45459 non-null  object 
 18  출판사리뷰        29156 non-null  object 
 19  INSE

In [ ]:
# 책 소개 없는 애들만
x = data.loc[data['책소개'].isna(), ['ISBN']]

In [ ]:
x.reset_index(drop=True,inplace=True)

In [ ]:
x['ISBN'] = x['ISBN'].astype('int64')
# 혹시 몰라서 int로. 32bit되려나

In [ ]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   ISBN    2998 non-null   int64
dtypes: int64(1)
memory usage: 23.6 KB


In [ ]:
def get_product_url_from_isbn(isbn):
    isbn_str = str(isbn)
    # ISBN을 URL의 keyword에 대입
    search_url = f"https://search.kyobobook.co.kr/search?keyword={isbn_str}&gbCode=TOT&target=total"
    response = requests.get(search_url)
    if response.status_code != 200:
        print(f"ISBN {isbn_str}: 상태 코드 {response.status_code}") # 에러날 시 에러코드 확인
        return None

    soup = BeautifulSoup(response.text, 'html.parser')

    # auto_overflow_wrap prod_name_group calss는 검색하는 ISBN이 없을 시 생성되지 않음 - 조건문으로 활용하여 None 반환
    container = soup.find("div", class_="auto_overflow_wrap prod_name_group")
    if container is None:
        print(f"ISBN {isbn_str}: 검색 결과 없음")
        return None

    # 생성된 페이지의 목표 태그(<a>---<href>)에서 URL만 추출 - 없으면 None 반환
    a_tag = container.find("a", href=True)
    if a_tag:
        product_url = a_tag.get('href')
        print(f"ISBN {isbn_str}: URL -> {product_url}")
        return product_url
    else:
        print(f"ISBN {isbn_str}: URL을 찾을 수 없음")
        return None

In [ ]:
x['url'] = x['ISBN'].apply(get_product_url_from_isbn)

ISBN 9788962515510: 상품 URL -> https://product.kyobobook.co.kr/detail/S000000969234
ISBN 9791186492369: 검색 결과 없음 (auto_overflow_wrap prod_name_group class 없음)
ISBN 9788928518357: 상품 URL -> https://product.kyobobook.co.kr/detail/S000201463394
ISBN 9791162431382: 상품 URL -> https://product.kyobobook.co.kr/detail/S000001812721
ISBN 9788915001275: 상품 URL -> https://product.kyobobook.co.kr/detail/S000202706008
ISBN 9788962397215: 상품 URL -> https://product.kyobobook.co.kr/detail/S000000966896
ISBN 9791137225138: 상품 URL -> https://product.kyobobook.co.kr/detail/S000060614084
ISBN 9788952785688: 상품 URL -> https://product.kyobobook.co.kr/detail/S000000734814
ISBN 9791137210981: 상품 URL -> https://product.kyobobook.co.kr/detail/S000060611981
ISBN 9791190145657: 상품 URL -> https://product.kyobobook.co.kr/detail/S000001936018
ISBN 9791127296377: 상품 URL -> https://product.kyobobook.co.kr/detail/S000060610785
ISBN 9791160455908: 검색 결과 없음 (auto_overflow_wrap prod_name_group class 없음)
ISBN 9791137267169: 

In [ ]:
file_path = "/content/drive/MyDrive/aiffel_final_project/data_renew/x_url.csv"

pd.DataFrame(x).to_csv(file_path, index=False)

In [ ]:
x['url'].isna().sum()

499

### 이미 존재하는 책 소개 row

In [ ]:
description = pd.read_csv("/content/drive/MyDrive/aiffel_final_project/data_renew/book_description_from_naver.csv")
x = pd.read_csv("/content/drive/MyDrive/aiffel_final_project/data_renew/x_url.csv")

In [ ]:
description

,ISBN,책소개
0,9791162431382,"임영석 시집 『나, 이제부터 삐딱하게 살기로 했다』는 크게 5부로 나누어져 있으며 ..."
1,9788962397215,▶ 이 책은 세법이론을 다룬 이론서입니다. 세법이론의 기초적이고 전반적인 내용을 학...
2,9788952785688,"엄마, 마녀가 나를 잡아먹으려고 해요! \n\n'네버랜드 마음이 자라는 성장 그림책..."
3,9791190145657,▶ 한국회화에 관한 내용을 담은 전문서적입니다.
4,9788982231032,"『무도와 무사도 인문학』은 〈일본 무도의 철학적 고찰〉, 〈무사의 마음, 일본의 국..."
...,...,...
868,9791168942882,말랑말랑 기분 좋은 치유계 EVERYDAY!\n\n가지런한 바가지 앞머리가 매력 포...
869,9791136412331,청라 퓨전 판타지 장편소설 『망작의 삼공자로 사는 법』 제3권. 북방의 군주라 불리...
870,9788942592791,▶ 이 책은 환경위해관리를 위한 노출평가를 다룬 이론서입니다. 환경위해관리를 위한 ...
871,9791137237254,태극권은 중국뿐만 아니라 전 세계적으로 많은 수련인구를 가지고 있다.\n\n수련생들...


In [ ]:
description.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 873 entries, 0 to 872
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ISBN    873 non-null    int64 
 1   책소개     873 non-null    object
dtypes: int64(1), object(1)
memory usage: 13.8+ KB


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ISBN         50000 non-null  int64  
 1   ITEM_ID      50000 non-null  int64  
 2   BID          42652 non-null  float64
 3   GOODS_NO     7348 non-null   float64
 4   분류           46967 non-null  object 
 5   제목           49996 non-null  object 
 6   부제           33406 non-null  object 
 7   원제           4506 non-null   object 
 8   저자           50000 non-null  object 
 9   발행자          50000 non-null  object 
 10  발행일          50000 non-null  object 
 11  페이지          50000 non-null  int64  
 12  가격           50000 non-null  int64  
 13  표지           50000 non-null  object 
 14  간략소개         48590 non-null  object 
 15  책소개          47002 non-null  object 
 16  저자소개         37934 non-null  object 
 17  목차           45459 non-null  object 
 18  출판사리뷰        29156 non-null  object 
 19  INSE

In [ ]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ISBN    2998 non-null   int64 
 1   url     2499 non-null   object
 2   책소개     873 non-null    object
 3   출판사리뷰   352 non-null    object
 4   분류      2895 non-null   object
 5   목차      1714 non-null   object
dtypes: int64(1), object(5)
memory usage: 140.7+ KB


In [ ]:
!pip install selenium
!apt-get update
!apt-get install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 26.4 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,351 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,714 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubun

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import random

#### 분류

In [ ]:
# Selenium 설정 (Colab 환경 등)
chrome_options = Options()
chrome_options.add_argument("--headless")           # 헤드리스 모드
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")

driver = webdriver.Chrome(options=chrome_options)

def extract_category(url):
    """
    주어진 URL에 대해 랜덤 딜레이 후 Selenium으로 상세페이지에 접속하고,
    BeautifulSoup을 사용해 카테고리 정보를 추출하여
    ">국내도서>시/에세이>한국시>현대시" 형태의 문자열을 반환합니다.
    """
    # 랜덤 딜레이 (3 ~ 7초)
    delay = random.uniform(3, 7)
    time.sleep(delay)

    driver.get(url)
    # 페이지 완전 로드를 위한 대기 (필요 시 조정)
    time.sleep(6)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    ul = soup.select_one('#scrollSpyProdInfo > div:nth-of-type(4) > div:nth-of-type(1) > ul')
    if ul:
        links = ul.select("a.intro_category_link")
        categories = [link.get_text(strip=True) for link in links]
        return ">" + ">".join(categories)
    else:
        return None

In [ ]:
# x['url']가 존재하고 x['분류']가 null인 행들만 선택
mask = x['url'].notnull() & x['분류'].isnull()
target_indices = x[mask].index.tolist()

print(f"전체 처리 대상 개수: {len(target_indices)}")

# 한 번에 3개씩 배치 처리 혹시 모를 트래픽
batch_size = 3
total = len(target_indices)

for i in range(0, total, batch_size):
    batch_indices = target_indices[i:i+batch_size]
    print(f"배치 {i//batch_size+1} 처리 중 (총 {len(batch_indices)}개)...")
    for idx in batch_indices:
        url = x.loc[idx, 'url']
        category_info = extract_category(url)
        x.loc[idx, '분류'] = category_info
        print(f"인덱스 {idx} | URL: {url} -> 분류: {category_info}")
    print(f"배치 {i//batch_size+1} 처리 완료, 다음 배치 전 7초 대기...")
    time.sleep(7)

driver.quit()

전체 처리 대상 개수: 88
배치 1 처리 중 (총 3개)...
인덱스 16 | URL: https://product.kyobobook.co.kr/detail/S000061695386 -> 분류: >국내도서>취업/수험서>전문직자격증>행정사
인덱스 31 | URL: https://product.kyobobook.co.kr/detail/S000000902976 -> 분류: >국내도서>정치/사회>법학>소송/판례>민사소송(법)>국내도서>정치/사회>대학교재>법학>국내도서>대학교재>정치/사회/법>법학
인덱스 75 | URL: https://product.kyobobook.co.kr/detail/S000000902909 -> 분류: >국내도서>과학>교양과학>교양생물>생물이야기
배치 1 처리 완료, 다음 배치 전 7초 대기...
배치 2 처리 중 (총 3개)...
인덱스 131 | URL: https://product.kyobobook.co.kr/detail/S000061425529 -> 분류: >국내도서>취업/수험서>전문직자격증>변리사
인덱스 141 | URL: https://product.kyobobook.co.kr/detail/S000001764248 -> 분류: >국내도서>기술/공학>의학>보건학>보건의료법규>국내도서>기술/공학>대학교재>의학>국내도서>대학교재>기술공학>의학
인덱스 147 | URL: https://product.kyobobook.co.kr/detail/S000061694344 -> 분류: >국내도서>정치/사회>법학>상법>특허/상표/지식재산/저작권
배치 2 처리 완료, 다음 배치 전 7초 대기...
배치 3 처리 중 (총 3개)...
인덱스 151 | URL: https://product.kyobobook.co.kr/detail/S000201274208 -> 분류: >국내도서>경제/경영>세무/회계>양도/소득/재산세>국내도서>경제/경영>대학교재>국내도서>대학교재>경제/경영
인덱스 154 | URL: https://ebook-product.kyobobook

In [ ]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ISBN    2998 non-null   int64 
 1   url     2499 non-null   object
 2   책소개     873 non-null    object
 3   출판사리뷰   352 non-null    object
 4   분류      2973 non-null   object
 5   목차      1714 non-null   object
dtypes: int64(1), object(5)
memory usage: 140.7+ KB


#### 책소개

In [ ]:
file_path = "/content/drive/MyDrive/aiffel_final_project/data_renew/x_url.csv"

pd.DataFrame(x).to_csv(file_path, index=False)

In [ ]:
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# Selenium 설정 (Chrome headless 모드)
chrome_options = Options()
chrome_options.add_argument("--headless")           # 헤드리스 모드
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
driver = webdriver.Chrome(options=chrome_options)

def extract_book_intro(url):
    """
    주어진 URL에 대해 랜덤 딜레이 후 Selenium으로 상세페이지에 접속하고,
    XPath 영역
       //*[@id="scrollSpyProdInfo"]/div[4]/div[2]
    와
       //*[@id="scrollSpyProdInfo"]/div[5]/div[2]
    내에서 클래스가 "info_text fw_bold" 또는 "info_text"인 모든 div 요소(또는 해당 요소 내의 텍스트 노드)를
    순차적으로 이어붙여 하나의 문자열로 반환합니다.

    만약 두 영역 모두에서 해당 클래스 요소가 전혀 없다면, None을 반환합니다.
    """
    # 랜덤 딜레이 (2 ~ 3초)
    time.sleep(random.uniform(1, 2))

    driver.get(url)
    # 페이지가 완전히 로드될 때까지 대기 (필요 시 조정)
    time.sleep(1)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # 두 영역 컨테이너 선택
    containers = []
    container4 = soup.select_one('#scrollSpyProdInfo > div:nth-of-type(4) > div:nth-of-type(2)')
    container5 = soup.select_one('#scrollSpyProdInfo > div:nth-of-type(5) > div:nth-of-type(2)')
    if container4:
        containers.append(container4)
    if container5:
        containers.append(container5)

    # 두 영역에서 "info_text" 관련 요소(예: "info_text fw_bold" 또는 "info_text")의 텍스트 노드 추출
    texts = []
    for cont in containers:
        divs = cont.find_all("div", class_=lambda c: c and "info_text" in c)
        if divs:
            for div in divs:
                texts.extend(list(div.stripped_strings))

    if texts:
        return " ".join(texts)
    else:
        return None

In [ ]:
# 재실행 시작 인덱스 지정 (예: 10부터 처리)
start_index = 217

# x DataFrame에 대해 x['url']가 존재하고 x['책소개']가 null인 행들 중에서 인덱스가 start_index 이상인 경우 선택
mask = x['url'].notnull() & x['책소개'].isnull() & (x.index >= start_index)
target_indices = x[mask].index.tolist()

print(f"전체 처리 대상 개수 (인덱스 {start_index}부터): {len(target_indices)}")

# 한 번에 5개씩 배치 처리 (트래픽 부담 완화)
batch_size = 10
total = len(target_indices)

for i in range(0, total, batch_size):
    batch_indices = target_indices[i:i+batch_size]
    print(f"배치 {i//batch_size+1} 처리 중 (총 {len(batch_indices)}개)...")
    for idx in batch_indices:
        url = x.loc[idx, 'url']
        intro_text = extract_book_intro(url)
        x.loc[idx, '책소개'] = intro_text
        print(f"인덱스 {idx} | URL: {url} -> 책소개: {intro_text}")
    print(f"배치 {i//batch_size+1} 처리 완료, 다음 배치 전 7초 대기...")
    time.sleep(1)

driver.quit()

전체 처리 대상 개수 (인덱스 217부터): 1535
배치 1 처리 중 (총 10개)...
인덱스 217 | URL: https://product.kyobobook.co.kr/detail/S000200551569 -> 책소개: 2020년 초부터 전 세계적으로 확산된 코로나는 우리나라 정치, 경제, 사회, 문화 등 모든 분야에 영향을 미쳤다. 거리두기 영향과 대인접촉 기피현상으로 재택근무가 늘어나고, 각종 모임과 행사 및 회식이 줄어드는 등 큰 사회적 변화를 겪었다. 코로나 영향으로 부동산 시장도 큰 영향을 받았다.
인덱스 219 | URL: https://product.kyobobook.co.kr/detail/S000201408366 -> 책소개: None
인덱스 220 | URL: https://product.kyobobook.co.kr/detail/S000201054243 -> 책소개: 이 책은 사회복지학을 다룬 이론서이다. 알면서도 몰랐던 장애의 기초적이고 전반적인 내용을 학습할 수 있도록 구성하였다.
인덱스 228 | URL: https://product.kyobobook.co.kr/detail/S000200152387 -> 책소개: None
인덱스 229 | URL: https://product.kyobobook.co.kr/detail/S000200367257 -> 책소개: 독자대상 : 경찰공무원 시험 준비생 구성 : 이론
인덱스 230 | URL: https://product.kyobobook.co.kr/detail/S000061695353 -> 책소개: 이 책은 소박한 민속과 민가의 향기 3를 다룬 한옥건축서적이다. 소박한 민속과 민가의 향기 3에 관한 기초적이고 전반적인 내용들이 수록되어 있다.
인덱스 236 | URL: https://product.kyobobook.co.kr/detail/S000001984653 -> 책소개: 『어도비 포토샵CC』는 〈포토샵 튜닝〉, 〈환경설정〉, 〈스크레치 디스크〉, 〈사진 촬영 기법과 포토샵〉, 〈이미지 해

In [ ]:
naru = pd.read_csv("/content/drive/MyDrive/aiffel_final_project/data_renew/aiffel_book_250306.csv")

In [ ]:
x = pd.read_csv("/content/drive/MyDrive/aiffel_final_project/data_renew/x_url.csv")

In [ ]:
naru.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ISBN         50000 non-null  int64  
 1   ITEM_ID      50000 non-null  int64  
 2   BID          42652 non-null  float64
 3   GOODS_NO     7348 non-null   float64
 4   분류           49206 non-null  object 
 5   제목           50000 non-null  object 
 6   부제           33406 non-null  object 
 7   원제           4506 non-null   object 
 8   저자           50000 non-null  object 
 9   발행자          50000 non-null  object 
 10  발행일          50000 non-null  object 
 11  페이지          50000 non-null  int64  
 12  가격           50000 non-null  int64  
 13  표지           50000 non-null  object 
 14  간략소개         48590 non-null  object 
 15  책소개          47891 non-null  object 
 16  저자소개         37934 non-null  object 
 17  목차           45459 non-null  object 
 18  출판사리뷰        29156 non-null  object 
 19  INSE

In [ ]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ISBN    2998 non-null   int64 
 1   url     2499 non-null   object
 2   책소개     2240 non-null   object
 3   출판사리뷰   352 non-null    object
 4   분류      2973 non-null   object
 5   목차      1714 non-null   object
dtypes: int64(1), object(5)
memory usage: 140.7+ KB


### 병합

In [ ]:
# '책소개' 컬럼이 존재하는지 확인
if '책소개' in naru.columns and '책소개' in x.columns:
    # ISBN을 기준으로 x를 딕셔너리로 변환 (빠른 조회를 위해)
    isbn_to_intro = x.set_index('ISBN')['책소개'].dropna().to_dict()

    # '책소개'가 None (또는 NaN)인 경우 x의 '책소개' 값으로 채우기
    naru['책소개'] = naru.apply(lambda row: isbn_to_intro.get(row['ISBN'], row['책소개']) if pd.isna(row['책소개']) else row['책소개'], axis=1)
else:
    print("'책소개' 컬럼이 존재하지 않습니다.")

In [ ]:
naru.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ISBN         50000 non-null  int64  
 1   ITEM_ID      50000 non-null  int64  
 2   BID          42652 non-null  float64
 3   GOODS_NO     7348 non-null   float64
 4   분류           49206 non-null  object 
 5   제목           50000 non-null  object 
 6   부제           33406 non-null  object 
 7   원제           4506 non-null   object 
 8   저자           50000 non-null  object 
 9   발행자          50000 non-null  object 
 10  발행일          50000 non-null  object 
 11  페이지          50000 non-null  int64  
 12  가격           50000 non-null  int64  
 13  표지           50000 non-null  object 
 14  간략소개         48590 non-null  object 
 15  책소개          49255 non-null  object 
 16  저자소개         37934 non-null  object 
 17  목차           45459 non-null  object 
 18  출판사리뷰        29156 non-null  object 
 19  INSE

In [ ]:
naru['책소개'] = naru.apply(lambda x: x['간략소개'] if pd.isna(x['책소개']) and pd.notna(x['간략소개']) else x['책소개'], axis=1)

In [ ]:
naru.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ISBN         50000 non-null  int64  
 1   ITEM_ID      50000 non-null  int64  
 2   BID          42652 non-null  float64
 3   GOODS_NO     7348 non-null   float64
 4   분류           49206 non-null  object 
 5   제목           50000 non-null  object 
 6   부제           33406 non-null  object 
 7   원제           4506 non-null   object 
 8   저자           50000 non-null  object 
 9   발행자          50000 non-null  object 
 10  발행일          50000 non-null  object 
 11  페이지          50000 non-null  int64  
 12  가격           50000 non-null  int64  
 13  표지           50000 non-null  object 
 14  간략소개         48590 non-null  object 
 15  책소개          49616 non-null  object 
 16  저자소개         37934 non-null  object 
 17  목차           45459 non-null  object 
 18  출판사리뷰        29156 non-null  object 
 19  INSE

In [ ]:
# '분류' 컬럼이 존재하는지 확인
if '분류' in naru.columns and '분류' in x.columns:
    # ISBN을 기준으로 x를 딕셔너리로 변환 (빠른 조회를 위해)
    isbn_to_intro = x.set_index('ISBN')['분류'].dropna().to_dict()

    # '분류'가 None (또는 NaN)인 경우 x의 '분류' 값으로 채우기
    naru['분류'] = naru.apply(lambda row: isbn_to_intro.get(row['ISBN'], row['분류']) if pd.isna(row['분류']) else row['분류'], axis=1)
else:
    print("'분류' 컬럼이 존재하지 않습니다.")

In [ ]:
naru.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ISBN         50000 non-null  int64  
 1   ITEM_ID      50000 non-null  int64  
 2   BID          42652 non-null  float64
 3   GOODS_NO     7348 non-null   float64
 4   분류           49243 non-null  object 
 5   제목           50000 non-null  object 
 6   부제           33406 non-null  object 
 7   원제           4506 non-null   object 
 8   저자           50000 non-null  object 
 9   발행자          50000 non-null  object 
 10  발행일          50000 non-null  object 
 11  페이지          50000 non-null  int64  
 12  가격           50000 non-null  int64  
 13  표지           50000 non-null  object 
 14  간략소개         48590 non-null  object 
 15  책소개          49255 non-null  object 
 16  저자소개         37934 non-null  object 
 17  목차           45459 non-null  object 
 18  출판사리뷰        29156 non-null  object 
 19  INSE

In [ ]:
file_path = "/content/drive/MyDrive/aiffel_final_project/data_renew/x_url.csv"
pd.DataFrame(x).to_csv(file_path, index=False)

In [ ]:
# original
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ISBN    2998 non-null   int64 
 1   url     2499 non-null   object
 2   책소개     873 non-null    object
 3   출판사리뷰   352 non-null    object
 4   분류      2895 non-null   object
 5   목차      1714 non-null   object
dtypes: int64(1), object(5)
memory usage: 140.7+ KB


In [ ]:
naru.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ISBN         50000 non-null  int64  
 1   ITEM_ID      50000 non-null  int64  
 2   BID          42652 non-null  float64
 3   GOODS_NO     7348 non-null   float64
 4   분류           49206 non-null  object 
 5   제목           50000 non-null  object 
 6   부제           33406 non-null  object 
 7   원제           4506 non-null   object 
 8   저자           50000 non-null  object 
 9   발행자          50000 non-null  object 
 10  발행일          50000 non-null  object 
 11  페이지          50000 non-null  int64  
 12  가격           50000 non-null  int64  
 13  표지           50000 non-null  object 
 14  간략소개         48590 non-null  object 
 15  책소개          49616 non-null  object 
 16  저자소개         37934 non-null  object 
 17  목차           45459 non-null  object 
 18  출판사리뷰        29156 non-null  object 
 19  INSE

In [ ]:
# 최종 데이터 저장
naru.to_csv("/content/drive/MyDrive/aiffel_final_project/data_renew/aiffel_book_250306_update(filled).csv", index=False)

* //*[@id="scrollSpyProdInfo"]/div[4]/div[3]/div/text()[1]
    * 해당 주소로 758개 다시 받아오기
* 출판사리뷰(서평) + 추천사
    * 존재하는 것만 빨리빨리 보고 넘어가게 코드 짜기
* 목차

In [ ]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ISBN    2998 non-null   int64 
 1   url     2499 non-null   object
 2   책소개     2240 non-null   object
 3   출판사리뷰   352 non-null    object
 4   분류      2973 non-null   object
 5   목차      1714 non-null   object
dtypes: int64(1), object(5)
memory usage: 140.7+ KB


In [ ]:
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# Selenium 설정 (Chrome headless 모드)
chrome_options = Options()
chrome_options.add_argument("--headless")           # 헤드리스 모드
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
driver = webdriver.Chrome(options=chrome_options)

def extract_book_intro(url):
    """
    주어진 URL에 대해 랜덤 딜레이 후 Selenium으로 상세페이지에 접속하고,
    XPath 영역
       //*[@id="scrollSpyProdInfo"]/div[4]/div[2]
    와
       //*[@id="scrollSpyProdInfo"]/div[5]/div[2]
    내에서 클래스가 "info_text fw_bold" 또는 "info_text"인 모든 div 요소(또는 해당 요소 내의 텍스트 노드)를
    순차적으로 이어붙여 하나의 문자열로 반환합니다.

    만약 두 영역 모두에서 해당 클래스 요소가 전혀 없다면, None을 반환합니다.
    """
    # 랜덤 딜레이 (2 ~ 3초)
    time.sleep(random.uniform(1, 2))

    driver.get(url)
    # 페이지가 완전히 로드될 때까지 대기 (필요 시 조정)
    time.sleep(1)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # 두 영역 컨테이너 선택
    containers = []
    container4 = soup.select_one('#scrollSpyProdInfo > div:nth-of-type(4) > div:nth-of-type(2)')
    container5 = soup.select_one('#scrollSpyProdInfo > div:nth-of-type(5) > div:nth-of-type(2)')
    if container4:
        containers.append(container4)
    if container5:
        containers.append(container5)

    # 두 영역에서 "info_text" 관련 요소(예: "info_text fw_bold" 또는 "info_text")의 텍스트 노드 추출
    texts = []
    for cont in containers:
        divs = cont.find_all("div", class_=lambda c: c and "info_text" in c)
        if divs:
            for div in divs:
                texts.extend(list(div.stripped_strings))

    if texts:
        return " ".join(texts)
    else:
        return None

In [ ]:
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# Selenium 설정 (Chrome headless 모드)
chrome_options = Options()
chrome_options.add_argument("--headless")           # 헤드리스 모드
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
driver = webdriver.Chrome(options=chrome_options)

def extract_book_intro(url):
    """
    주어진 URL에 대해 랜덤 딜레이 후 Selenium으로 상세페이지에 접속하고,
    XPath 영역:
       //*[@id="scrollSpyProdInfo"]/div[4]/div[2]
    와
       //*[@id="scrollSpyProdInfo"]/div[5]/div[2]
    내에서 클래스가 "info_text fw_bold" 또는 "info_text"인 모든 div 요소(또는 해당 요소 내의 텍스트 노드)를
    순차적으로 이어붙여 하나의 문자열로 반환합니다.

    추가로, XPath 영역:
       //*[@id="scrollSpyProdInfo"]/div[4]/div[3]/div/text()
    혹은
       //*[@id="scrollSpyProdInfo"]/div[4]/div[3]/div/text()[1]
    에 해당하는 숫자 또는 텍스트 노드도 함께 추출하여 결과에 포함시킵니다.

    만약 모든 영역에서 해당 텍스트가 전혀 없다면, None을 반환합니다.
    """
    # 랜덤 딜레이 (1 ~ 2초)
    time.sleep(random.uniform(1, 2))

    driver.get(url)
    # 페이지가 완전히 로드될 때까지 대기 (필요 시 조정)
    time.sleep(1)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # 기존 컨테이너 2개 선택
    containers = []
    container4 = soup.select_one('#scrollSpyProdInfo > div:nth-of-type(4) > div:nth-of-type(2)')
    container5 = soup.select_one('#scrollSpyProdInfo > div:nth-of-type(5) > div:nth-of-type(2)')
    if container4:
        containers.append(container4)
    if container5:
        containers.append(container5)

    # 추가 컨테이너: div[4]/div[3]
    container_extra = soup.select_one('#scrollSpyProdInfo > div:nth-of-type(4) > div:nth-of-type(3)')
    if container_extra:
        containers.append(container_extra)

    # 각 컨테이너 내에서 "info_text" 관련 요소(예: "info_text fw_bold" 또는 "info_text") 및
    # 추가 컨테이너의 경우, 모든 텍스트 노드를 추출
    texts = []
    for cont in containers:
        # 만약 컨테이너에 'info_text' 클래스가 포함된 div가 있다면
        divs = cont.find_all("div", class_=lambda c: c and "info_text" in c)
        if divs:
            for div in divs:
                texts.extend(list(div.stripped_strings))
        else:
            # 'info_text' 클래스가 없으면, 컨테이너 내의 모든 텍스트 노드를 추출
            texts.extend(list(cont.stripped_strings))

    if texts:
        return " ".join(texts)
    else:
        return None

In [ ]:
# 재실행 시작 인덱스 지정 (예: 10부터 처리)
start_index = 1

# x DataFrame에 대해 x['url']가 존재하고 x['책소개']가 null인 행들 중에서 인덱스가 start_index 이상인 경우 선택
mask = x['url'].notnull() & x['책소개'].isnull() & (x.index >= start_index)
target_indices = x[mask].index.tolist()

print(f"전체 처리 대상 개수 (인덱스 {start_index}부터): {len(target_indices)}")

# 한 번에 5개씩 배치 처리 (트래픽 부담 완화)
batch_size = 10
total = len(target_indices)

for i in range(0, total, batch_size):
    batch_indices = target_indices[i:i+batch_size]
    print(f"배치 {i//batch_size+1} 처리 중 (총 {len(batch_indices)}개)...")
    for idx in batch_indices:
        url = x.loc[idx, 'url']
        intro_text = extract_book_intro(url)
        x.loc[idx, '책소개'] = intro_text
        print(f"인덱스 {idx} | URL: {url} -> 책소개: {intro_text}")
    print(f"배치 {i//batch_size+1} 처리 완료, 다음 배치 전 7초 대기...")
    time.sleep(1)

driver.quit()

전체 처리 대상 개수 (인덱스 1부터): 301
배치 1 처리 중 (총 10개)...
인덱스 6 | URL: https://product.kyobobook.co.kr/detail/S000060614084 -> 책소개: 찜하기 장바구니 저자(글) 최정아 어릴 적부터 용돈으로 동네서점에서 시리즈를 사모으던 떡잎부터 책수집러. 그림책 애호가 / 수집가 10년차 초등학교 교사 월간지 <좋은교사> '내 마음 속 그림책 산책' 기고 <기독교사대회> 선택강의 강사 <좋은교사> 자율연수 강사 <기윤실 수련회> 선택강의 강사 교원학습공동체 연수 강사 zzong323@naver.com 펼치기
인덱스 8 | URL: https://product.kyobobook.co.kr/detail/S000060611981 -> 책소개: 찜하기 장바구니 저자(글) 키키
인덱스 10 | URL: https://product.kyobobook.co.kr/detail/S000060610785 -> 책소개: 찜하기 장바구니 저자(글) 강현영 그녀는 일산에 거주중이며 이 책은 그녀의 1.5번째 책이다. 펼치기
인덱스 12 | URL: https://product.kyobobook.co.kr/detail/S000060620237 -> 책소개: 찜하기 장바구니 저자(글) 2021 공도초 5학년 2반 친구들
인덱스 13 | URL: https://product.kyobobook.co.kr/detail/S000003945769 -> 책소개: 펼치기 원서번역서 원
인덱스 18 | URL: https://product.kyobobook.co.kr/detail/S000061442350 -> 책소개: 찜하기 장바구니 저자(글) 구한솔 "구한솔" 1996년 출생. 서울 서초구 사람. 2017년 명지대학교 영화뮤지컬학부에 입학했으나 수 년 간 휴학했다. 2017년 「경상북도 영상콘텐츠 시나리오 공모전」에 시나리오 <동쪽 끝 변호사 선생>이 우수상에 당선되어 작품 활동을 시작했다. 펼치기
인덱스 29 | URL: http

In [ ]:
x = pd.read_csv("/content/drive/MyDrive/aiffel_final_project/data_renew/x_url.csv")

In [ ]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ISBN    2998 non-null   int64 
 1   url     2499 non-null   object
 2   책소개     2240 non-null   object
 3   출판사리뷰   352 non-null    object
 4   분류      2973 non-null   object
 5   목차      1714 non-null   object
dtypes: int64(1), object(5)
memory usage: 140.7+ KB


In [ ]:
# x.iloc[827] = "<잃은 사람들의 만찬> 작가 강태근 장편소설. 한국 사회의 구조적 아픔, 그 슬픈 자화상을 구체적 이야기로 직시하고 이를 넘어설 방략을 모색하는 작품이다."

# 인덱스 43 | URL: https://product.kyobobook.co.kr/detail/S000061695042
# 인덱스 73 | URL: https://product.kyobobook.co.kr/detail/S000200419232
# 인덱스 82 | URL: https://product.kyobobook.co.kr/detail/S000200497027
# 인덱스 100 | URL: https://product.kyobobook.co.kr/detail/S000201040684
# 인덱스 107 | URL: https://product.kyobobook.co.kr/detail/S000200417698
# 인덱스 110 | URL: https://product.kyobobook.co.kr/detail/S000061777727
# 인덱스 126 | URL: https://product.kyobobook.co.kr/detail/S000200354622
# 인덱스 219 | URL: https://product.kyobobook.co.kr/detail/S000201408366
# 인덱스 228 | URL: https://product.kyobobook.co.kr/detail/S000200152387
# 인덱스 247 | URL: https://product.kyobobook.co.kr/detail/S000200616446
# 인덱스 266 | URL: https://product.kyobobook.co.kr/detail/S000200322124
# 인덱스 280 | URL: https://product.kyobobook.co.kr/detail/S000200490503
# 인덱스 287 | URL: https://product.kyobobook.co.kr/detail/S000200312124
# 인덱스 302 | URL: https://product.kyobobook.co.kr/detail/S000201884950
# 인덱스 311 | URL: https://product.kyobobook.co.kr/detail/S000061897658
# 인덱스 314 | URL: https://product.kyobobook.co.kr/detail/S000200369661
# 인덱스 316 | URL: https://product.kyobobook.co.kr/detail/S000200403146
# 인덱스 346 | URL: https://product.kyobobook.co.kr/detail/S000200410168
# 인덱스 350 | URL: https://product.kyobobook.co.kr/detail/S000200490340
# 인덱스 356 | URL: https://product.kyobobook.co.kr/detail/S000200365320
# 인덱스 387 | URL: https://product.kyobobook.co.kr/detail/S000003945888
# 인덱스 388 | URL: https://product.kyobobook.co.kr/detail/S000201649879
# 인덱스 418 | URL: https://product.kyobobook.co.kr/detail/S000200490271
# 인덱스 438 | URL: https://product.kyobobook.co.kr/detail/S000202610250
# 인덱스 439 | URL: https://product.kyobobook.co.kr/detail/S000200419226
# 인덱스 450 | URL: https://product.kyobobook.co.kr/detail/S000200242343
# 인덱스 464 | URL: https://product.kyobobook.co.kr/detail/S000201373944
# 인덱스 469 | URL: https://product.kyobobook.co.kr/detail/S000200069946
# 인덱스 473 | URL: https://product.kyobobook.co.kr/detail/S000200473472
# 인덱스 478 | URL: https://product.kyobobook.co.kr/detail/S000200477584
# 인덱스 484 | URL: https://product.kyobobook.co.kr/detail/S000200344912
# 인덱스 485 | URL: https://product.kyobobook.co.kr/detail/S000000524541
# 인덱스 492 | URL: https://product.kyobobook.co.kr/detail/S000202623779
# 인덱스 497 | URL: https://product.kyobobook.co.kr/detail/S000061696181
# 인덱스 514 | URL: https://product.kyobobook.co.kr/detail/S000200365334
# 인덱스 518 | URL: https://product.kyobobook.co.kr/detail/S000001656557
# 인덱스 526 | URL: https://product.kyobobook.co.kr/detail/S000202406307
# 인덱스 527 | URL: https://product.kyobobook.co.kr/detail/S000061695195
# 인덱스 560 | URL: https://product.kyobobook.co.kr/detail/S000200419224
# 인덱스 561 | URL: https://product.kyobobook.co.kr/detail/S000200396524
# 인덱스 627 | URL: https://product.kyobobook.co.kr/detail/S000061757407
# 인덱스 648 | URL: https://product.kyobobook.co.kr/detail/S000200384808
# 인덱스 653 | URL: https://product.kyobobook.co.kr/detail/S000061863291
# 인덱스 655 | URL: https://product.kyobobook.co.kr/detail/S000202031540
# 인덱스 696 | URL: https://product.kyobobook.co.kr/detail/S000200417729
# 인덱스 699 | URL: https://product.kyobobook.co.kr/detail/S000200747399
# 인덱스 721 | URL: https://product.kyobobook.co.kr/detail/S000200396790
# 인덱스 775 | URL: https://product.kyobobook.co.kr/detail/S000215561399
# 인덱스 805 | URL: https://product.kyobobook.co.kr/detail/S000201400228
# 인덱스 827 | URL: https://product.kyobobook.co.kr/detail/S000001769612 엄마없는작가새끼
# 인덱스 861 | URL: https://product.kyobobook.co.kr/detail/S000200144282
# 인덱스 874 | URL: https://product.kyobobook.co.kr/detail/S000213447330
# 인덱스 907 | URL: https://product.kyobobook.co.kr/detail/S000200152041
# 인덱스 909 | URL: https://product.kyobobook.co.kr/detail/S000200314837
# 인덱스 943 | URL: https://product.kyobobook.co.kr/detail/S000200490279
# 인덱스 960 | URL: https://ebook-product.kyobobook.co.kr/dig/epd/ebook/E000005154406 - 작품소개에
# 인덱스 1056 | URL: https://product.kyobobook.co.kr/detail/S000200493185
# 인덱스 1076 | URL: https://product.kyobobook.co.kr/detail/S000200305083
# 인덱스 1102 | URL: https://product.kyobobook.co.kr/detail/S000202257455
# 인덱스 1159 | URL: https://product.kyobobook.co.kr/detail/S000200738608
# 인덱스 1166 | URL: https://product.kyobobook.co.kr/detail/S000200404955
# 인덱스 1200 | URL: https://product.kyobobook.co.kr/detail/S000061757137
# 인덱스 1207 | URL: https://product.kyobobook.co.kr/detail/S000061583989
# 인덱스 1210 | URL: https://product.kyobobook.co.kr/detail/S000201978234
# 인덱스 1255 | URL: https://product.kyobobook.co.kr/detail/S000001732406
# 인덱스 1259 | URL: https://product.kyobobook.co.kr/detail/S000200348080
# 인덱스 1298 | URL: https://product.kyobobook.co.kr/detail/S000200493187
# 인덱스 1305 | URL: https://product.kyobobook.co.kr/detail/S000200417749
# 인덱스 1406 | URL: https://product.kyobobook.co.kr/detail/S000062210884
# 인덱스 1472 | URL: https://product.kyobobook.co.kr/detail/S000200484689
# 인덱스 1475 | URL: https://product.kyobobook.co.kr/detail/S000201264808
# 인덱스 1485 | URL: https://product.kyobobook.co.kr/detail/S000201978049
# 인덱스 1571 | URL: https://product.kyobobook.co.kr/detail/S000200497024
# 인덱스 1618 | URL: https://product.kyobobook.co.kr/detail/S000202042951
# 인덱스 1630 | URL: https://product.kyobobook.co.kr/detail/S000061350222 김봉씨발럼
# 인덱스 1645 | URL: https://product.kyobobook.co.kr/detail/S000201556848
# 인덱스 1654 | URL: https://product.kyobobook.co.kr/detail/S000200771744
# 인덱스 1658 | URL: https://product.kyobobook.co.kr/detail/S000202752849 그냥 한글자
# 인덱스 1667 | URL: https://product.kyobobook.co.kr/detail/S000200403142
# 인덱스 1682 | URL: https://product.kyobobook.co.kr/detail/S000201595240
# 인덱스 1688 | URL: https://product.kyobobook.co.kr/detail/S000200493184
# 인덱스 1726 | URL: https://product.kyobobook.co.kr/detail/S000200608232
# 인덱스 1747 | URL: https://product.kyobobook.co.kr/detail/S000200363807
# 인덱스 1755 | URL: https://product.kyobobook.co.kr/detail/S000200320650
# 인덱스 1774 | URL: https://product.kyobobook.co.kr/detail/S000200363902
# 인덱스 1780 | URL: https://product.kyobobook.co.kr/detail/S000201884961
# 인덱스 1784 | URL: https://product.kyobobook.co.kr/detail/S000200747394
# 인덱스 1813 | URL: https://product.kyobobook.co.kr/detail/S000200227305
# 인덱스 1817 | URL: https://product.kyobobook.co.kr/detail/S000201393305
# 인덱스 1822 | URL: https://product.kyobobook.co.kr/detail/S000200367281
# 인덱스 1840 | URL: https://product.kyobobook.co.kr/detail/S000200355678
# 인덱스 1855 | URL: https://product.kyobobook.co.kr/detail/S000200367277
# 인덱스 1868 | URL: https://product.kyobobook.co.kr/detail/S000200419223
# 인덱스 1879 | URL: https://product.kyobobook.co.kr/detail/S000201416586
# 인덱스 1894 | URL: https://product.kyobobook.co.kr/detail/S000061694903
# 인덱스 1912 | URL: https://product.kyobobook.co.kr/detail/S000200396523
# 인덱스 2011 | URL: https://product.kyobobook.co.kr/detail/S000201301615
# 인덱스 2022 | URL: https://product.kyobobook.co.kr/detail/S000200410147
# 인덱스 2023 | URL: https://product.kyobobook.co.kr/detail/S000200608188
# 인덱스 2040 | URL: https://product.kyobobook.co.kr/detail/S000061350257
# 인덱스 2090 | URL: https://product.kyobobook.co.kr/detail/S000200355677
# 인덱스 2104 | URL: https://product.kyobobook.co.kr/detail/S000201301783
# 인덱스 2119 | URL: https://product.kyobobook.co.kr/detail/S000201549826
# 인덱스 2130 | URL: https://product.kyobobook.co.kr/detail/S000200488989
# 인덱스 2144 | URL: https://product.kyobobook.co.kr/detail/S000200419225
# 인덱스 2146 | URL: https://product.kyobobook.co.kr/detail/S000200367282
# 인덱스 2149 | URL: https://product.kyobobook.co.kr/detail/S000200419222
# 인덱스 2214 | URL: https://product.kyobobook.co.kr/detail/S000061757921
# 인덱스 2220 | URL: https://product.kyobobook.co.kr/detail/S000001942428 한글자
# 인덱스 2234 | URL: https://product.kyobobook.co.kr/detail/S000201345317 한글자
# 인덱스 2271 | URL: https://product.kyobobook.co.kr/detail/S000200918275
# 인덱스 2273 | URL: https://product.kyobobook.co.kr/detail/S000200460931 한글자
# 인덱스 2326 | URL: https://product.kyobobook.co.kr/detail/S000200419229
# 인덱스 2351 | URL: https://product.kyobobook.co.kr/detail/S000202036275
# 인덱스 2383 | URL: https://product.kyobobook.co.kr/detail/S000200411419
# 인덱스 2392 | URL: https://product.kyobobook.co.kr/detail/S000201453614
# 인덱스 2435 | URL: https://product.kyobobook.co.kr/detail/S000200490501
# 인덱스 2458 | URL: https://product.kyobobook.co.kr/detail/S000201583352 한글자자
# 인덱스 2471 | URL: https://product.kyobobook.co.kr/detail/S000200616426
# 인덱스 2513 | URL: https://product.kyobobook.co.kr/detail/S000001647721 -불량 글로벌 기업과 악덕 국가들이 판을 치는 21세기. 그 판을 뒤엎고자 그가 돌아왔다─! 인피니티 인더스트리의 창업자 백미르! 그가 주도하는 새로운 신기술들로 무장한 새로운 시대! 이제 그 시대의 중심은... 바로 우리들이다!
# 인덱스 2515 | URL: https://product.kyobobook.co.kr/detail/S000200396518
# 인덱스 2534 | URL: https://product.kyobobook.co.kr/detail/S000200496807
# 인덱스 2596 | URL: https://product.kyobobook.co.kr/detail/S000200419237
# 인덱스 2619 | URL: https://product.kyobobook.co.kr/detail/S000200355671
# 인덱스 2622 | URL: https://product.kyobobook.co.kr/detail/S000201416597
# 인덱스 2630 | URL: https://product.kyobobook.co.kr/detail/S000061532216
# 인덱스 2640 | URL: https://product.kyobobook.co.kr/detail/S000200497029 아가페
# 인덱스 2650 | URL: https://product.kyobobook.co.kr/detail/S000201495859
# 인덱스 2681 | URL: https://product.kyobobook.co.kr/detail/S000061350259
# 인덱스 2729 | URL: https://product.kyobobook.co.kr/detail/S000200363776
# 인덱스 2730 | URL: https://product.kyobobook.co.kr/detail/S000200608235
# 인덱스 2740 | URL: https://product.kyobobook.co.kr/detail/S000200411396
# 인덱스 2744 | URL: https://product.kyobobook.co.kr/detail/S000061696001
# 인덱스 2759 | URL: https://product.kyobobook.co.kr/detail/S000202342917
# 인덱스 2764 | URL: https://product.kyobobook.co.kr/detail/S000200355673
# 인덱스 2805 | URL: https://product.kyobobook.co.kr/detail/S000200059070 한글자
# 인덱스 2806 | URL: https://product.kyobobook.co.kr/detail/S000200384858
# 인덱스 2807 | URL: https://product.kyobobook.co.kr/detail/S000201301786
# 인덱스 2812 | URL: https://product.kyobobook.co.kr/detail/S000202688421
# 인덱스 2833 | URL: https://product.kyobobook.co.kr/detail/S000200488952
# 인덱스 2840 | URL: https://product.kyobobook.co.kr/detail/S000201416603
# 인덱스 2873 | URL: https://product.kyobobook.co.kr/detail/S000200179060
# 인덱스 2878 | URL: https://product.kyobobook.co.kr/detail/S000200381921
# 인덱스 2889 | URL: https://product.kyobobook.co.kr/detail/S000200367280
# 인덱스 2895 | URL: https://product.kyobobook.co.kr/detail/S000200115820 한글자
# 인덱스 2933 | URL: https://product.kyobobook.co.kr/detail/S000200616440
# 인덱스 2956 | URL: https://product.kyobobook.co.kr/detail/S000200488979
# 인덱스 2960 | URL: https://product.kyobobook.co.kr/detail/S000201309320 한글자
# 인덱스 2975 | URL: https://product.kyobobook.co.kr/detail/S000202150498
# 인덱스 2988 | URL: https://product.kyobobook.co.kr/detail/S000200031397

### 출판사 서평 - url

In [6]:
data = pd.read_csv("/content/drive/MyDrive/aiffel_final_project/data_renew/aiffel_book_250306_update(filled).csv")

In [ ]:
# 출판사리뷰 없는 애들만
x = data.loc[data['출판사리뷰'].isna(), ['ISBN']]

In [ ]:
def get_product_url_from_isbn(isbn):
    isbn_str = str(isbn)
    # ISBN을 URL의 keyword에 대입
    search_url = f"https://search.kyobobook.co.kr/search?keyword={isbn_str}&gbCode=TOT&target=total"
    response = requests.get(search_url)
    if response.status_code != 200:
        print(f"ISBN {isbn_str}: 상태 코드 {response.status_code}") # 에러날 시 에러코드 확인
        return None

    soup = BeautifulSoup(response.text, 'html.parser')

    # auto_overflow_wrap prod_name_group calss는 검색하는 ISBN이 없을 시 생성되지 않음 - 조건문으로 활용하여 None 반환
    container = soup.find("div", class_="auto_overflow_wrap prod_name_group")
    if container is None:
        print(f"ISBN {isbn_str}: 검색 결과 없음")
        return None

    # 생성된 페이지의 목표 태그(<a>---<href>)에서 URL만 추출 - 없으면 None 반환
    a_tag = container.find("a", href=True)
    if a_tag:
        product_url = a_tag.get('href')
        print(f"ISBN {isbn_str}: URL -> {product_url}")
        return product_url
    else:
        print(f"ISBN {isbn_str}: URL을 찾을 수 없음")
        return None

In [ ]:
# 저장 파일 경로 설정
save_dir = "/content/drive/MyDrive/aiffel_final_project/data_renew"
save_file = os.path.join(save_dir, "x_review.csv")

# 이미 저장된 결과가 있는지 확인 (끊긴 부분부터 재시작)
if os.path.exists(save_file):
    df_saved = pd.read_csv(save_file)
    processed_isbns = set(df_saved["ISBN"].astype(str))
    print(f"저장된 결과가 있습니다. 처리된 ISBN 수: {len(processed_isbns)}")
else:
    df_saved = pd.DataFrame(columns=["ISBN", "product_url"])
    processed_isbns = set()

# 이미 처리된 결과를 포함하여 결과 DataFrame 초기화
results = df_saved.copy()
batch_count = 0  # 이번 실행에서 새로 처리한 건수

# 전체 x DataFrame의 ISBN에 대해 반복 (이미 처리된 것은 건너뜀)
for idx, row in x.iterrows():
    isbn = row["ISBN"]
    if str(isbn) in processed_isbns:
        continue

    url = get_product_url_from_isbn(isbn)
    # 새로운 행 DataFrame 생성 후 결과와 합치기
    new_row = pd.DataFrame([{"ISBN": isbn, "product_url": url}])
    results = pd.concat([results, new_row], ignore_index=True)
    processed_isbns.add(str(isbn))
    batch_count += 1

    # 300건마다 저장
    if batch_count % 300 == 0:
        results.to_csv(save_file, index=False)
        print(f"새로 처리한 {batch_count}건 저장 완료. (전체 저장된 건수: {len(results)})")
        time.sleep(1)  # 저장 후 잠시 대기

# 최종 저장
results.to_csv(save_file, index=False)
print("모든 ISBN 처리 완료 및 최종 저장됨.")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
ISBN 9791159629655: URL -> https://product.kyobobook.co.kr/detail/S000001782446
ISBN 9791163899525: URL -> https://product.kyobobook.co.kr/detail/S000001823740
ISBN 9788967641757: URL -> https://product.kyobobook.co.kr/detail/S000001048840
ISBN 9791155775844: URL -> https://product.kyobobook.co.kr/detail/S000001728969
ISBN 9791164410507: URL -> https://product.kyobobook.co.kr/detail/S000001828758
ISBN 9791159422232: URL -> https://product.kyobobook.co.kr/detail/S000001778123
ISBN 9791136264121: URL -> https://product.kyobobook.co.kr/detail/S000001712203
ISBN 9788989625315: URL -> https://product.kyobobook.co.kr/detail/S000001428808
ISBN 9791164451968: URL -> https://product.kyobobook.co.kr/detail/S000001829837
ISBN 9791138012072: URL -> https://product.kyobobook.co.kr/detail/S000001716127
ISBN 9791136234704: URL -> https://product.kyobobook.co.kr/detail/S000001711212
ISBN 9791168488670: URL -> https://product.kyobobook.co.kr/detail/S000001853613
ISBN

### 출판사리뷰 - text

In [7]:
data.columns

Index(['ISBN', 'ITEM_ID', 'BID', 'GOODS_NO', '분류', '제목', '부제', '원제', '저자',
       '발행자', '발행일', '페이지', '가격', '표지', '간략소개', '책소개', '저자소개', '목차', '출판사리뷰',
       'INSERT_DATE', 'UPDATE_DATE', 'len_cat'],
      dtype='object')

In [20]:
# pd.set_option('display.max_rows', None)  # 모든 행 출력
# pd.set_option('display.max_columns', None)  # 모든 열 출력
pd.set_option('display.max_colwidth', None)  # 컬럼 내용 길이 제한 해제
pd.set_option('display.expand_frame_repr', False)  # 줄 바꿈 없이 한 줄로 출력

In [21]:
data.loc[(data['출판사리뷰'].isna() == False),['출판사리뷰']]

,출판사리뷰
0,"평범했지만 내가 주인공이었던 영화 같은 시간들\n어느 날 문득 곁으로 다가온 인생의 명장면들을 기록하다\n\n“저요, 사는 게 뭔지 진짜 궁금해졌어요. 그 안에 영화도 있어요.” \n영화 [찬실이는 복도 많지]에는 이런 대사가 나온다. 영화에 젊은 날을 다 바쳤으나, 결국 영화 때문에 모든 것을 잃고, 그럼에도 또다시 영화에서 삶의 의미를 찾은 찬실이. 어떤 일에 열정과 진심을 다했던 일, 사람, 꿈과의 관계에서 행복을 얻고, 때로는 실망하지만 또다시 관계를 맺어나가는 과정들은 참 평범하지만, 영화 같다. 우리의 인생처럼.\n\n영화는 아주 가까이에 있다. 타인의 이야기라고 생각하며 무심하게 재생한 영화의 주인공은 인종도 성별도 사는 곳도 다르지만, 때때로 우리의 이야기가 된다. 영화는 평범한 사람들의 살아가는 이야기를 담아내고, 우리는 영화를 통해 삶을 되돌아보기 때문이다. 영화를 사랑하는 사람들이 자신의 인생 영화로 꼽는 영화들은 대체로 소박하다. [리틀 포레스트], [패터슨], [벌새], [소공녀] 등 잔잔한 흐름 안에 가슴을 쿵 하고 울리는 장면들이 있다. 일러스트레이터 무궁화 작가는 『인생에서 정지 버튼 누르고 싶었던 순간들』을 쓰고 그리며 영화 속에서 자신을 되돌아보게 된 순간들, 독자들도 스스로 성장할 수 있는 이야기들을 담았다. \n\n돌이켜볼 수 있다는 추억이 있다는 건 행복한 일이야\n나를 가장 나답게 만들어주었던 마이 페이보릿 시퀀스!\n\n“난 내가 싫어질 때 그 마음을 들여다 봐. \n아 지금 내가 나를 사랑할 수 없구나.”_ 영화 [벌새] 중\n\n본래 영화에 관심이 없었던 무궁화 작가가 영화를 그리기 시작한 것은 영화가 내 이야기로 느껴지기 시작한 순간부터였다. 남들 다 하는 취업 준비는 안 하고 자신이 좋아하는 족구에 열정을 쏟는 [족구왕]의 만섭이를 보면서 작가 또한 취업 준비 대신에 그림에 열중할 수 있는 용기를 얻었다. [걸어도 걸어도]의 료타를 보면서 엄마의 부탁을 미루고 있는 자신을 반성했고, [우리들]의 지아와 선을 보면서 대학 시절 절친했던 친구와 멀어진 관계를 이해하고 과거의 자신과 화해하게 되었다. 작가에게 영화는 고민을 털어놓는 상담소였고, 관계를 돌아보는 거울이었으며, 자신이 진정으로 무엇을 좋아하는지 말할 수 있는 기회이기도 했다. 영화는 그렇게 작가가 현재를 되돌아보고, 스스로의 삶을 영화의 한 장면처럼 만들어나가는 힘을 준 것이다. \n\n이 책에는 총 26편의 영화 명장면을 담은 일러스트와 에세이가 담겨 있다. 친구들과 자연에서 얻은 재료로 요리한 맛있는 음식을 먹으면서 도시에서 지친 마음을 달래는 [리틀 포레스트]의 혜원, 지루하게 반복되는 하루 속에서 일상의 사소한 변화들을 담아내며 시를 써내려가는 [패터슨]의 패터슨, 그리고 타인과는 다른 선택을 하고 방황을 하면서도 자신의 가치관을 지키기 위해서 고군분투하는 [소공녀]의 미소까지. 많은 사람이 인생 영화로 꼽는 영화들의 명장면을 통해 우리의 사소한 일상을 특별하게 그려낸 이야기들을 담았다. 작가가 담아낸 그림과 글, 영화 속 명대사들을 읽다 보면 나만의 시퀀스를 발견하고 행복한 순간을 다시금 떠올리는 계기가 될 것이다."
2,"책을 읽고 글을 쓰며 성장을 꿈꾸는 사람들의 진솔한 자기고백\n\n지금 이 순간을 살아가는 사람이라면 누구든 다음과 같은 고민을 갖고 있을 것이다.\n\n“오늘도…내일도…똑같은 제자리걸음의 삶을 살고 있는 것은 아닐까?”\n“나는 정말로 내가 원하는 나의 모습으로 살아가고 있을까?”\n“한 치 앞도 파악하기 어려운 불확실성 속에서 내 미래는 어떻게 전개될까?”\n\n특히 세상이 말하는 ‘어른’이 된 지 얼마 되지도 않은 것 같은데 취업하고, 결혼하고, 자녀를 낳아 양육하며 숨 가쁘게 일상을 살아 나가야 하는 사람들. 나를 돌아볼 여유도 없이 가정을 위해 모든 것을 희생하며 하루하루를 지내는 이 땅의 30대 후반~40대의 어머니, 아버지들이라면 누구나 이러한 고민을 해본 적이 있을 것이다.\n\n이 책 『책에 나를 바치다』는 책과 사람을 통해 그렇게 꼭꼭 숨겨 놓은 고민을 풀어 놓고, 공감 받고 공감해 주며, 사색과 긍정으로 순화하여 지속적인 성장을 꿈꾸는 사람들의 진솔한 자기고백이자 성장의 일기다.\n\n이 책에 참여한 ‘책·바·침’은 2018년 학부모들의 작은 책 읽기 프로젝트에서 시작하여 지금까지 평택 지역을 중심으로 활발한 활동을 전개 중인 독서모임이다. 책·바·침의 이름으로 참여한 아홉 명의 작가들은 나이도, 성별도, 현재의 위치도 각기 다르지만 쳇바퀴 돌 듯 같은 곳을 반복하는 일상에서 벗어나 성장하고 싶어 하고, 동시에 자신과 가족, 우리 사회에게 더 나은 미래를 위한 청사진을 독서를 통해 찾아 나가고 있는 사람들이다. \n책을 함께 읽고, 고민을 서로 듣고 눈물을 흘려주고, 언제나 긍정적인 대화로 서로를 충전하며 독서, 필사, 토론, 미라클모닝 훈련, 긍정훈련 등의 자기계발 기법을 통해 서로 간에 선한 영향력을 전파하는 것이 책·바·침 멤버들의 성장 모습이기도 하다.\n\n9명의 저자들이 39개의 이야기를 통해 들려주는 고민과 성장의 이야기는 어쩌면 작고 평범한 우리 모두의 이야기일지도 모른다. 하지만 오로지 앞만 보고 달려왔던 자신을 되돌아보고, 평행선을 달리듯 서로 이해할 수 없다고 생각해 왔던 남편, 아내, 자식과의 관계의 해법을 모색하고, 생활고와 격무 속에서도 자기 자신만을 위한 작은 노력을 시도하는 모습들. 이러한 모습들은 극한 경쟁 속에서 지쳐가는 현대 사회의 많은 이들에게 ‘나도 책을 통해서 변할 수 있다!’는 작지만 큰 희망을 선사해 줄 것이다.\n\n나의 변화는 책 덕분이었다(오현옥)\n넘어지면 누군가의 손을 잡아야만 일어날 수 있다고 의지했던 과거의 모습은 사라졌다. 다른 사람의 말에 휘둘리지 않고 진정한 나의 모습으로 살아가는 엄마이자 꿈 작가로서의 제2의 인생을 꿈꾼다.\n\n이제는, 나답게 살자(여동호)\n나는 나일뿐, 그 누구도 될 수 없다. 흉내만 내면 내 안의 조각품은 모조품이 될 뿐이다. 흉내만 내는 글을 쓰다, 이젠 손때 묻은 나의 글을 쓰려고 한다. 내 방식대로.\n\n독서가 필요해(김혜중)\n50대에 들어서며 나의 소망을 ‘성장’이라고 정했다. 성공을 바라보기엔 숨이 가쁘다는 느낌도 커서였겠지만, 이미 지나간 인생에서 이루어 놓은 게 없었기 때문이다. 자가발전 할 수 있는 게 무엇일까 하고 고민하던 종착점에서 찾은 결론이 책이었고 독서 모임이었다. 독서는 어떤 모습으로 나를 이끌고 성장시킬지 따라가 보련다.\n\n나약한 나도 무엇이든 도전할 수 있다(우기숙)\n난 항상 혼자이고 할 수 없는 게 많은 사람이라고 생각했다. 독서모임을 하면서 많은 책을 접하고 내가 할 수 있는 일, 내 주위 사람들을 둘러보게 된다. 나같이 어려움을 겪는 분들을 보면 조금이나마 도움(정보)을 주려고 노력을 한다.\n\n지금 행복하자(이어은)\n마흔에 책이라곤 안 읽던 내가 독서 모임을 시작했고 독서 모임을 하다 보니 좋아하는 것을 하면서 살고 싶어졌다. 행복하고 즐겁게 살고 싶어 ‘어은쓰 TV’란 이름으로 유튜브까지 시작하게 됐다. \n난 요즘 신세계에 빠져 있다.\n\n서툴러도 괜찮아(이유정)\n30살이 넘으면 단단하고 지혜로운 여자가 되어 있을 줄 알았다. 하지만 34살의 난 여전히 삶 속에서 쉽게 좌절하고 흔들린다. 뒤늦게 독서를 통해 서툴러도 조금씩 성장하고 있는 자신을 발견한다. 오늘도 어제보다 성장할 나에게 응원과 격려를 보낸다. \n\n독서로 제2의 인생을 살다(김진수)\n32살 때부터 독서를 하고, 36살 때부터 글을 쓰고, 37살 때부터 책을 썼다. 이제 독서 나이 10살, 글쓰기 나이 6살, 책 쓰기 나이 5살이 되어 보니 삶이란 것이 어떤 것인지 조금은 알게 되었다. 책에 나를 바쳐본 사람은 안다. 책이 나에게 주는 놀라운 힘을 말이다. \n\n사람은 무엇으로 변하는가?(임은희)\n별 의미도 없이 반복되는 따분한 나날에, 독서는 차츰 나의 삶을 즐기는 법을 가르쳐 주었다. 나 자신의 삶을 소중히 여길 줄 아는, 이미 전과는 다른 사람이 되어 있는 나!\n\nDream teller(정해광)\n현재 취미로 통기타를 치는 40대 직

In [22]:
pd.reset_option('all')

<ipython-input-22-056908e7e3ef>:1: FutureWarning: data_manager option is deprecated and will be removed in a future version. Only the BlockManager will be available.
  pd.reset_option('all')
<ipython-input-22-056908e7e3ef>:1: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.reset_option('all')


In [2]:
x = pd.read_csv("/content/drive/MyDrive/aiffel_final_project/data_renew/x_review.csv")

In [3]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20844 entries, 0 to 20843
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ISBN         20844 non-null  int64 
 1   product_url  18937 non-null  object
dtypes: int64(1), object(1)
memory usage: 325.8+ KB


In [5]:
x['product_url'].iloc[:5]

,product_url
0,https://product.kyobobook.co.kr/detail/S000001...
1,https://product.kyobobook.co.kr/detail/S000061...
2,https://product.kyobobook.co.kr/detail/S000001...
3,https://product.kyobobook.co.kr/detail/S000001...
4,NaN


In [2]:
!pip install selenium
!apt-get update
!apt-get install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 29.6 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,774 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,369 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.

In [17]:
import os
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# CSV 저장 경로 설정
save_path = "/content/drive/MyDrive/aiffel_final_project/data_renew/x_publish_review.csv"

# CSV 파일이 존재하면 불러오기, 없으면 기존 데이터셋을 사용(예: original_data.csv로부터)
if os.path.exists(save_path):
    x = pd.read_csv(save_path)
    print(f"CSV 파일 로드됨: {save_path}")
else:
    # 여기에 원본 데이터를 불러오는 코드를 작성합니다.
    # 예: x = pd.read_csv("/content/drive/MyDrive/aiffel_final_project/data/original_data.csv")
    raise FileNotFoundError(f"CSV 파일이 {save_path}에 존재하지 않습니다. 원본 데이터를 먼저 불러오세요.")

# Selenium 설정 (Chrome headless 모드)
chrome_options = Options()
chrome_options.add_argument("--headless")           # 헤드리스 모드
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
driver = webdriver.Chrome(options=chrome_options)

def extract_book_details(url):
    """
    주어진 URL에 접속 후, '추천사'와 '출판사리뷰' 데이터를 추출하여 반환합니다.

    추천사:
      - 'product_detail_area book_recommend' 클래스가 존재하면,
        <ul class="recommend_list"> 내부의 각 <li class="recommend_item">에서
        추천 매체명과 추천 텍스트를 "매체명 : 추천 텍스트" 형식으로 추출 후 줄바꿈(\n)으로 연결.
      - 없으면 None 반환.

    출판사리뷰:
      - 'product_detail_area book_publish_review' 클래스가 존재하면,
        내부의 <p class="info_text">의 텍스트를 추출.
      - 없으면 None 반환.
    """
    # 랜덤 딜레이 (1 ~ 2초)
    time.sleep(random.uniform(1, 2))

    driver.get(url)
    time.sleep(1)  # 페이지 로드 대기
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # 추천사 추출
    recommend_text = None
    recommend_section = soup.find("div", class_="product_detail_area book_recommend")
    if recommend_section:
        ul = recommend_section.find("ul", class_="recommend_list")
        if ul:
            rec_items = ul.find_all("li", class_="recommend_item")
            rec_list = []
            for item in rec_items:
                source_tag = item.find("a", class_="title_heading fc_spot type_link")
                source = source_tag.get_text(strip=True) if source_tag else ""
                info_tag = item.find("p", class_="info_text")
                info = info_tag.get_text(strip=True) if info_tag else ""
                if source or info:
                    rec_list.append(f"{source} : {info}")
            if rec_list:
                recommend_text = "\n".join(rec_list)

    # 출판사리뷰 추출
    publish_review_text = None
    publish_section = soup.find("div", class_="product_detail_area book_publish_review")
    if publish_section:
        p_tag = publish_section.find("p", class_="info_text")
        if p_tag:
            publish_review_text = p_tag.get_text(separator="\n", strip=True)

    return recommend_text, publish_review_text

CSV 파일 로드됨: /content/drive/MyDrive/aiffel_final_project/data_renew/x_publish_review.csv


In [ ]:
# 시작 인덱스 지정 (원하는 경우 조정)
start_index = 4326

# '추천사'와 '출판사리뷰' 컬럼이 없으면 추가
if '추천사' not in x.columns:
    x['추천사'] = None
if '출판사리뷰' not in x.columns:
    x['출판사리뷰'] = None

# product_url이 존재하고, 아직 '추천사'와 '출판사리뷰'가 채워지지 않은 행 선택 (start_index 이상)
mask = x['product_url'].notnull() & x['추천사'].isnull() & x['출판사리뷰'].isnull() & (x.index >= start_index)
target_indices = x[mask].index.tolist()

print(f"전체 처리 대상 개수 (인덱스 {start_index}부터): {len(target_indices)}")

batch_size = 10
total = len(target_indices)
processed_count = 0

for i in range(0, total, batch_size):
    batch_indices = target_indices[i:i+batch_size]
    print(f"배치 {i//batch_size+1} 처리 중 (총 {len(batch_indices)}개)...")
    for idx in batch_indices:
        url = x.loc[idx, 'product_url']
        rec_text, pub_review = extract_book_details(url)
        x.loc[idx, '추천사'] = rec_text
        x.loc[idx, '출판사리뷰'] = pub_review
        processed_count += 1
        print(f"인덱스 {idx} | URL: {url} -> 추천사: {rec_text}, 출판사리뷰: {pub_review}")

        # 300번 처리마다 CSV 파일 저장
        if processed_count % 300 == 0:
            x.to_csv(save_path, index=False)
            print(f"Processed {processed_count} rows. CSV 저장됨: {save_path}")

    print(f"배치 {i//batch_size+1} 처리 완료, 다음 배치 전 1초 대기...")
    time.sleep(1)

# 최종 저장
x.to_csv(save_path, index=False)
print(f"모든 데이터 처리 완료. CSV 저장됨: {save_path}")

driver.quit()

전체 처리 대상 개수 (인덱스 4326부터): 15018
배치 1 처리 중 (총 10개)...
인덱스 4326 | URL: https://product.kyobobook.co.kr/detail/S000200313130 -> 추천사: None, 출판사리뷰: None
인덱스 4327 | URL: https://product.kyobobook.co.kr/detail/S000001039439 -> 추천사: None, 출판사리뷰: None
인덱스 4328 | URL: https://product.kyobobook.co.kr/detail/S000201279358 -> 추천사: 유선식
                                ((전)인천광역시강화교육지원청 교육장) : “글이나 그림으로 자신을 표현할 수 있다는 것은 커다란 능력이다. 여기 『강화 아이들이 만든 두근두근 강화 이야기』 그림책이 있다. 이 책에는 강화 합일초등학교 학생들이 가진 배움의 힘과 미래를 주도적으로 만들어 나갈 수 있는 역량이 담겨 있다.”
조선미
                                (인천광역시교육청 세계시민교육과 장학관) : “이토록 멋진 우리 마을! 한 사람 한 사람의 자기 주체성을 형성하는 일, 자신을 만들어가는 일 혹은 살아가는 힘을 키우는 일은 자신과 가까운 사람과 함께, 자신과 가까운 곳에서 더 효과적으로 이루어질 수 있다. 내 주변의 가까운 이웃과 내가 살고 있는 지역은 배움의 도반이 된다. 학생들에게 마을은 배움의 공간이다. 마을을 발견하고 새롭게 만들어가는 일이 이 그림책에서 펼쳐지고 있다.”
한상준
                                (사우디아라비아 젯다한국국제학교장) : “‘눈길 함부로 걷지 마라, 오늘 걸어간 발자국이 뒷사람의 여정이 될지니.’ 마을 교육운동의 성지 ‘강화’에서 선도적으로 마을 교육활동을 실천해 온 합일초 선생님들과 마을 공동체의 주인공으로 우뚝 선 합일초 어린이들이 만든 『강화 아이들이 만든 두근두근 강화

In [24]:
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# Selenium 설정 (Chrome headless 모드)
chrome_options = Options()
chrome_options.add_argument("--headless")           # 헤드리스 모드
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
driver = webdriver.Chrome(options=chrome_options)

def extract_book_details(url):
    """
    주어진 URL에 접속 후, '추천사'와 '출판사리뷰' 데이터를 추출하여 반환합니다.

    추천사:
      - 'product_detail_area book_recommend' 클래스가 존재하면,
        <ul class="recommend_list"> 내부의 각 <li class="recommend_item">에서
        추천 매체명과 추천 텍스트를 "매체명 : 추천 텍스트" 형식으로 추출 후 줄바꿈(\n)으로 연결.
      - 없으면 None 반환.

    출판사리뷰:
      - 'product_detail_area book_publish_review' 클래스가 존재하면,
        내부의 <p class="info_text">의 텍스트를 추출.
      - 없으면 None 반환.
    """
    # 랜덤 딜레이 (1 ~ 2초)
    time.sleep(random.uniform(1, 2))

    driver.get(url)
    time.sleep(1)  # 페이지 로드 대기
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # 추천사 추출
    recommend_text = None
    recommend_section = soup.find("div", class_="product_detail_area book_recommend")
    if recommend_section:
        ul = recommend_section.find("ul", class_="recommend_list")
        if ul:
            rec_items = ul.find_all("li", class_="recommend_item")
            rec_list = []
            for item in rec_items:
                source_tag = item.find("a", class_="title_heading fc_spot type_link")
                source = source_tag.get_text(strip=True) if source_tag else ""
                info_tag = item.find("p", class_="info_text")
                info = info_tag.get_text(strip=True) if info_tag else ""
                if source or info:
                    rec_list.append(f"{source} : {info}")
            if rec_list:
                recommend_text = "\n".join(rec_list)

    # 출판사리뷰 추출
    publish_review_text = None
    publish_section = soup.find("div", class_="product_detail_area book_publish_review")
    if publish_section:
        p_tag = publish_section.find("p", class_="info_text")
        if p_tag:
            publish_review_text = p_tag.get_text(separator="\n", strip=True)

    return recommend_text, publish_review_text

In [ ]:
# 시작 인덱스 지정 (예: 0부터)
start_index = 0

# x DataFrame에 '추천사'와 '출판사리뷰' 컬럼 추가 (없으면)
if '추천사' not in x.columns:
    x['추천사'] = None
if '출판사리뷰' not in x.columns:
    x['출판사리뷰'] = None

# product_url이 존재하고, 아직 '추천사'와 '출판사리뷰'가 채워지지 않은 행 선택 (start_index 이상)
mask = x['product_url'].notnull() & x['추천사'].isnull() & x['출판사리뷰'].isnull() & (x.index >= start_index)
target_indices = x[mask].index.tolist()

print(f"전체 처리 대상 개수 (인덱스 {start_index}부터): {len(target_indices)}")

batch_size = 10
total = len(target_indices)
processed_count = 0

for i in range(0, total, batch_size):
    batch_indices = target_indices[i:i+batch_size]
    print(f"배치 {i//batch_size+1} 처리 중 (총 {len(batch_indices)}개)...")
    for idx in batch_indices:
        url = x.loc[idx, 'product_url']
        rec_text, pub_review = extract_book_details(url)
        x.loc[idx, '추천사'] = rec_text
        x.loc[idx, '출판사리뷰'] = pub_review
        processed_count += 1
        print(f"인덱스 {idx} | URL: {url} -> 추천사: {rec_text}, 출판사리뷰: {pub_review}")

        # 300번 처리마다 CSV 파일 저장
        if processed_count % 300 == 0:
            save_path = "/content/drive/MyDrive/aiffel_final_project/data_renew/x_publish_review.csv"
            x.to_csv(save_path, index=False)
            print(f"Processed {processed_count} rows. CSV 저장됨: {save_path}")

    print(f"배치 {i//batch_size+1} 처리 완료, 다음 배치 전 1초 대기...")
    time.sleep(1)

driver.quit()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
배치 101 처리 완료, 다음 배치 전 1초 대기...
배치 102 처리 중 (총 10개)...
인덱스 1110 | URL: https://product.kyobobook.co.kr/detail/S000001827700 -> 추천사: None, 출판사리뷰: None
인덱스 1111 | URL: https://product.kyobobook.co.kr/detail/S000001835113 -> 추천사: None, 출판사리뷰: None
인덱스 1112 | URL: https://product.kyobobook.co.kr/detail/S000001039390 -> 추천사: None, 출판사리뷰: None
인덱스 1113 | URL: https://product.kyobobook.co.kr/detail/S000001173064 -> 추천사: None, 출판사리뷰: None
인덱스 1114 | URL: https://product.kyobobook.co.kr/detail/S000001949908 -> 추천사: None, 출판사리뷰: None
인덱스 1115 | URL: https://product.kyobobook.co.kr/detail/S000061695315 -> 추천사: None, 출판사리뷰: None
인덱스 1116 | URL: https://product.kyobobook.co.kr/detail/S000200087257 -> 추천사: None, 출판사리뷰: None
인덱스 1117 | URL: https://product.kyobobook.co.kr/detail/S000200485279 -> 추천사: None, 출판사리뷰: None
인덱스 1118 | URL: https://product.kyobobook.co.kr/detail/S000000903003 -> 추천사: None, 출판사리뷰: None
인덱스 1119 | URL: https://product.kyobobook.co.kr/detail/